In [1]:
from gbdxtools import Interface
from gbdxtools import CatalogImage
from gbdxtools.task import env
import json

gbdx = Interface()

In [2]:
catalog_id = env.inputs.get('catalog_id', '1030010050CDCB00')
bbox = env.inputs.get('bbox', '-8.870814331642114, 7.9966215609953295, -8.85557865965, 8.00185723299')

#image = CatalogImage(catalog_id, bbox=map(float, bbox.split(",")))
#image.plot(w=10, h=10)

In [ ]:
metadata = image.ipe.metadata['image']
#print(json.dumps(metadata, indent=4))

In [4]:
data_url = 's3://receiving-dgcs-tdgplatform-com/057761542010_01_003'

In [5]:
ao_task = gbdx.Task('AOP_Strip_Processor',
                    data=data_url,
                    bands='MS',
                    dra_mode='BaseLayerMatch')
ao_task.impersonation_allowed = True

In [6]:
catalog_id = env.inputs.get('catalog_id', '1030010050CDCB00')
image = CatalogImage(catalog_id, band_type="MS")
imgBounds = image.bounds

chosenBounds = [-8.914697,6.999527,-8.709932,7.204651]
print('image bounds', imgBounds)
print('chose bounds', chosenBounds)

bounds = [max(imgBounds[0], chosenBounds[0]), max(imgBounds[1], chosenBounds[1]),
                min(imgBounds[2], chosenBounds[2]), min(imgBounds[3], chosenBounds[3])]

print('fixed bounds', bounds)

('image bounds', (-8.87079712, 6.989294710474309, -8.719214807747036, 8.03119975))
('chose bounds', [-8.914697, 6.999527, -8.709932, 7.204651])
('fixed bounds', [-8.87079712, 6.999527, -8.719214807747036, 7.204651])


In [7]:
# problem: ship value may not be set
clip_task = gbdx.Task('gdalcrop',
                     image=ao_task.outputs.data.value,
                     #image='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/lt_ao_bs_match1/',
                     crop='{} {} {} {}'.format(bounds[0], bounds[1], bounds[2], bounds[3]),
                     ship="false")

In [8]:
tile_task = gbdx.Task('splitter-gbdx:0.0.5',
                     data_in=clip_task.outputs.cropped.value)

In [9]:
fe_task = gbdx.Task('fe-gbdx:0.0.9',
                   data_in=tile_task.outputs.data_out.value)

In [21]:
clustervis_task = gbdx.Task('clustervis-gbdx:0.0.14',
                           data_in=tile_task.outputs.data_out.value,
                           features_in=fe_task.outputs.data_out.value,
                           #data_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/tiles2',
                           #features_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/fe-out2',
                           train='false')

In [22]:
#tasks = [ao_task]
#tasks2 = [clip_task]
#tasks = [ao_task, clip_task]
#tasks = [clip_task, tile_task]
#tasks = [fe_task]
#tasks = [clustervis_task]
#tasks = [clip_task, tile_task, fe_task, clustervis_task]
tasks = [ao_task, clip_task, tile_task, fe_task, clustervis_task]
workflow = gbdx.Workflow(tasks)

In [23]:
workflow.savedata(ao_task.outputs.data, location='liberia-test-acomp')
workflow.savedata(clip_task.outputs.cropped, location='liberia-test-clip')
workflow.savedata(tile_task.outputs.data_out, location='liberia-test-tiles')
workflow.savedata(fe_task.outputs.data_out, location='liberia-test-fe')
workflow.savedata(clustervis_task.outputs.data_out, location='liberia-test-clustervis')

In [24]:
workflow.execute()

u'4836997554487050785'

In [28]:
print workflow.id
print workflow.status
#print(json.dumps(workflow.events[len(workflow.events)-1], indent=4))
#print(json.dumps(workflow.events, indent=4))
for e in workflow.events:
    print("    {} {}".format(e['task'], e['event']))
    #print(e)
print(json.dumps(workflow.stdout, indent=4))
print(json.dumps(workflow.stderr, indent=4))

4836997554487050785
{u'state': u'complete', u'event': u'succeeded'}
    AOP_Strip_Processor_fcab7cae submitted
    gdalcrop_0db919a6 submitted
    splitter-gbdx_0.0.5_a489b4e8 submitted
    fe-gbdx_0.0.9_efdd322a submitted
    clustervis-gbdx_0.0.14_c514f50d submitted
    AOP_Strip_Processor_fcab7cae scheduled
    AOP_Strip_Processor_fcab7cae started
    AOP_Strip_Processor_fcab7cae succeeded
    gdalcrop_0db919a6 scheduled
    gdalcrop_0db919a6 started
    gdalcrop_0db919a6 succeeded
    splitter-gbdx_0.0.5_a489b4e8 scheduled
    splitter-gbdx_0.0.5_a489b4e8 started
    splitter-gbdx_0.0.5_a489b4e8 succeeded
    fe-gbdx_0.0.9_efdd322a scheduled
    fe-gbdx_0.0.9_efdd322a started
    fe-gbdx_0.0.9_efdd322a succeeded
    clustervis-gbdx_0.0.14_c514f50d scheduled
    clustervis-gbdx_0.0.14_c514f50d started
    clustervis-gbdx_0.0.14_c514f50d succeeded
[
    {
        "stdout": "Tiles Input dir: /mnt/work/input/data_in\nFeatures Input dir: /mnt/work/input/data_in\nTrain: False\nOutput dir

[
    {
        "stderr": "/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.\n  \"This module will be removed in 0.20.\", DeprecationWarning)\n/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.\n  warnings.warn(msg, category=DeprecationWarning)\n", 
        "id": "4836997554492981085", 
        "name": "clustervis-gbdx_0.0.14_c514f50d", 
        "taskType": "clustervis-gbdx:0.0.14"
    }, 
    {
        "s

In [1]:
#workflow.cancel()

In [ ]:
catalog_id = env.inputs.get('catalog_id', '1030010050CDCB00')
image = CatalogImage(catalog_id, band_type="MS")

In [ ]:
image

In [ ]:
imgBounds = image.bounds

In [ ]:
metadata = image.ipe.metadata['image']
print(json.dumps(metadata, indent=4))

In [ ]:
clip_task.outputs.cropped.value